In [1]:
import pandas as pd
import numpy as np

### groupby 집계 

In [2]:
# 샘플 데이터프레임 생성
idx = ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'C','D', 'D', 'D', 'D', 'E', 'E', 'E']
col = ['col1', 'col2', 'col3']
data = np.random.randint(0,9, (15,3))
df1 = pd.DataFrame(data=data, index=idx, columns = col).reset_index()
df1

,index,col1,col2,col3
0,A,6,1,1
1,A,5,1,8
2,B,2,5,8
3,B,6,6,5
4,B,8,1,8
5,C,3,6,1
6,C,1,1,7
7,C,8,0,4
8,D,4,7,2
9,D,6,6,1


In [3]:
# groupby() : 특정 컬럼별 통계치 구하기
df1.groupby('index').mean()  # sum() 합계 | mean() 평균 | count() 개수 | val() 분산 | std() 표준편차 | max() 최대값 | min() 최소값

,col1,col2,col3
index,,,
A,5.500000,1.000000,4.50
B,5.333333,4.000000,7.00
C,4.000000,2.333333,4.00
D,4.750000,6.000000,2.75
E,5.000000,3.666667,6.00


In [4]:
# 둘 이상의 통계치 동시에 구하기
df1.groupby('index').agg(['sum', 'mean'])

col1           col2           col3      
       sum      mean  sum      mean  sum  mean
index                                         
A       11  5.500000    2  1.000000    9  4.50
B       16  5.333333   12  4.000000   21  7.00
C       12  4.000000    7  2.333333   12  4.00
D       19  4.750000   24  6.000000   11  2.75
E       15  5.000000   11  3.666667   18  6.00

In [5]:
# 둘 이상의 통계치 동시에 구할때 함수 각각 적용하기
df1.groupby('index').agg({'col1' : 'mean', 'col2' : 'median', 'col3' : ['var', 'size']})

col1   col2       col3     
           mean median        var size
index                                 
A      5.500000    1.0  24.500000    2
B      5.333333    5.0   3.000000    3
C      4.000000    1.0   9.000000    3
D      4.750000    6.0   4.916667    4
E      5.000000    2.0   3.000000    3

In [6]:
# 둘 이상의 통계치 동시에 구할 때 컬럼의 멀티인덱스 정리, 소수점도 정리
def flatten_cols(df):
    df.columns = [' / '.join(x) for x in df.columns.to_flat_index()]
    return df

df1.groupby('index').agg(['sum', 'mean']).pipe(flatten_cols).round(2)

,col1 / sum,col1 / mean,col2 / sum,col2 / mean,col3 / sum,col3 / mean
index,,,,,,
A,11,5.50,2,1.00,9,4.50
B,16,5.33,12,4.00,21,7.00
C,12,4.00,7,2.33,12,4.00
D,19,4.75,24,6.00,11,2.75
E,15,5.00,11,3.67,18,6.00


In [7]:
# 하나의 대표값이 아닌 여러개의 값을 데이터프레임으로 호출
df1.groupby('index').describe().T

index              A         B         C         D         E
col1 count  2.000000  3.000000  3.000000  4.000000  3.000000
     mean   5.500000  5.333333  4.000000  4.750000  5.000000
     std    0.707107  3.055050  3.605551  0.957427  1.732051
     min    5.000000  2.000000  1.000000  4.000000  3.000000
     25%    5.250000  4.000000  2.000000  4.000000  4.500000
     50%    5.500000  6.000000  3.000000  4.500000  6.000000
     75%    5.750000  7.000000  5.500000  5.250000  6.000000
     max    6.000000  8.000000  8.000000  6.000000  6.000000
col2 count  2.000000  3.000000  3.000000  4.000000  3.000000
     mean   1.000000  4.000000  2.333333  6.000000  3.666667
     std    0.000000  2.645751  3.214550  0.816497  3.785939
     min    1.000000  1.000000  0.000000  5.000000  1.000000
     25%    1.000000  3.000000  0.500000  5.750000  1.500000
     50%    1.000000  5.000000  1.000000  6.000000  2.000000
     75%    1.000000  5.500000  3.500000  6.250000  5.000000
     max    1.000000  6.000000  6.000000  7.000000  8.000000
col3 count  2.000000  3.000000  3.000000  4.000000  3.000000
     mean   4.500000  7.000000  4.000000  2.750000  6.000000
     std    4.949747  1.732051  3.000000  2.217356  1.732051
     min    1.000000  5.000000  1.000000  1.000000  4.000000
     25%    2.750000  6.500000  2.500000  1.750000  5.500000
     50%    4.500000  8.000000  4.000000  2.000000  7.000000
     75%    6.250000  8.000000  5.500000  3.000000  7.000000
     max    8.000000  8.000000  7.000000  6.000000  7.000000

### 사용자정의 함수 활용/집계

In [8]:
# apply 메서드 활용, 각 그룹별 col1 상위 2위만 추출
def top(df1, n=2, col='col1'):
    return df1.sort_values(by=col)[-n:]
df1.groupby('index', group_keys=False).apply(top)

,index,col1,col2,col3
1,A,5,1,8
0,A,6,1,1
3,B,6,6,5
4,B,8,1,8
5,C,3,6,1
7,C,8,0,4
10,D,5,5,2
9,D,6,6,1
13,E,6,2,7
14,E,6,8,7


In [9]:
# 조금더 쉽게
def get_top2(x):
    return x.sort_values('col1').head(2)
df1.groupby('index').apply(get_top2)

index  col1  col2  col3
index                           
A     1      A     5     1     8
      0      A     6     1     1
B     2      B     2     5     8
      3      B     6     6     5
C     6      C     1     1     7
      5      C     3     6     1
D     8      D     4     7     2
      11     D     4     6     6
E     12     E     3     1     4
      13     E     6     2     7

In [10]:
# lambda 로도 가능
df1.groupby('index').apply(lambda x:x.sort_values('col1').head(2))

index  col1  col2  col3
index                           
A     1      A     5     1     8
      0      A     6     1     1
B     2      B     2     5     8
      3      B     6     6     5
C     6      C     1     1     7
      5      C     3     6     1
D     8      D     4     7     2
      11     D     4     6     6
E     12     E     3     1     4
      13     E     6     2     7

In [11]:
# 기존 집계함수 대신 사용자 정의 함수 적용
def iqr_func(x):
    q3, q1 = np.percentile(x, [75, 25])
    return q3 - q1
df1.groupby('index').agg(iqr_func)

,col1,col2,col3
index,,,
A,0.50,0.0,3.50
B,3.00,2.5,1.50
C,3.50,3.0,3.00
D,1.25,0.5,1.25
E,1.50,3.5,1.50


### 필터 및 변환

In [12]:
# filter() : 그룹화한 데이터에서 원하는 데이터 걸러냄
df1.groupby('index').filter(lambda x : x['index'].count() > 2)

,index,col1,col2,col3
2,B,2,5,8
3,B,6,6,5
4,B,8,1,8
5,C,3,6,1
6,C,1,1,7
7,C,8,0,4
8,D,4,7,2
9,D,6,6,1
10,D,5,5,2
11,D,4,6,6


In [13]:
# get_group() : 그룹화한 데이터에서 해당 데이터만 추출, loc랑 뭐가 다른건지?
df1.groupby('index').get_group('B')

,index,col1,col2,col3
2,B,2,5,8
3,B,6,6,5
4,B,8,1,8


In [14]:
# transform() : 데이터를 표준화할 뿐 종전 기존 데이터프레임 크기 유지
df2 = df1.copy()
df2['col1_mean'] = df2.groupby('index').col1.transform('mean')
df2

,index,col1,col2,col3,col1_mean
0,A,6,1,1,5.500000
1,A,5,1,8,5.500000
2,B,2,5,8,5.333333
3,B,6,6,5,5.333333
4,B,8,1,8,5.333333
5,C,3,6,1,4.000000
6,C,1,1,7,4.000000
7,C,8,0,4,4.000000
8,D,4,7,2,4.750000
9,D,6,6,1,4.750000


### groupby 인덱스 처리

In [15]:
# 기존 인덱스 유지, 아래의 reset_index() 붙인 거와 동일 (그래서 별 의미 없을수도)
df1.groupby('index', as_index=False).sum()

,index,col1,col2,col3
0,A,11,2,9
1,B,16,12,21
2,C,12,7,12
3,D,19,24,11
4,E,15,11,18


In [16]:
df1.groupby('index').sum().reset_index()

,index,col1,col2,col3
0,A,11,2,9
1,B,16,12,21
2,C,12,7,12
3,D,19,24,11
4,E,15,11,18


### Na 처리

In [17]:
# null 포함 샘플 데이터프레임 생성
df3 = df1.copy()
df3.loc[6, 'index'] = np.NaN
df3.loc[9, 'col1'] = np.NaN
df3

,index,col1,col2,col3
0,A,6.0,1,1
1,A,5.0,1,8
2,B,2.0,5,8
3,B,6.0,6,5
4,B,8.0,1,8
5,C,3.0,6,1
6,NaN,1.0,1,7
7,C,8.0,0,4
8,D,4.0,7,2
9,D,NaN,6,1


In [18]:
# 기본적으로 Nan 값은 집계함수에서 제외
df3.groupby('index').mean()

,col1,col2,col3
index,,,
A,5.500000,1.000000,4.50
B,5.333333,4.000000,7.00
C,5.500000,3.000000,2.50
D,4.333333,6.000000,2.75
E,5.000000,3.666667,6.00


In [19]:
# 하지만 dropna=False 인 경우 집계 컬럼의 NaN 포함되어 계산 (산출이 되는 컬럼에는 변동 없음)
df3.groupby('index', dropna=False).mean()

,col1,col2,col3
index,,,
A,5.500000,1.000000,4.50
B,5.333333,4.000000,7.00
C,5.500000,3.000000,2.50
D,4.333333,6.000000,2.75
E,5.000000,3.666667,6.00
NaN,1.000000,1.000000,7.00


### 그룹별 각각 조회

In [20]:
# 그룹별로 각각 순회하며 조회
for idx, group in df1.groupby('index'):
    display(group.head())

,index,col1,col2,col3
0,A,6,1,1
1,A,5,1,8


,index,col1,col2,col3
2,B,2,5,8
3,B,6,6,5
4,B,8,1,8


,index,col1,col2,col3
5,C,3,6,1
6,C,1,1,7
7,C,8,0,4


,index,col1,col2,col3
8,D,4,7,2
9,D,6,6,1
10,D,5,5,2
11,D,4,6,6


,index,col1,col2,col3
12,E,3,1,4
13,E,6,2,7
14,E,6,8,7
